<a href="https://colab.research.google.com/github/bda-lab/KGembedding/blob/main/ElEm_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train-Valid Split for n-Ball Model

In [ ]:
SAVE_PATH="{}.{}"
FILE_NAME="hasAnnotation42.owl"  #name of finalized ontology (.owl)

file_path = FILE_NAME
  #print(SPLIT)
file = {}
  # count = {}
for k in ['train', 'valid', 'test']:

    #file[k] = open(SAVE_PATH.format(DATASET, SPLIT, k, "txt"), 'w')
    file[k] = open(SAVE_PATH.format( k, "txt"), 'w')
    # count[k] = 0
file['train_norm'] = open(SAVE_PATH.format( "train_norm", "owl"), 'w')
  # count['train_norm'] = 0
subclass_axioms = []
other_axioms = []
co=0
train = 0
valid = 0
test = 0
classes = {}
relations = {}
subclass_classes = {}
  # First pass: Get all classes in subclass axioms, number of axioms
with open(file_path) as f:
    for line in f:
      co+=1
      og_line = line
      if line.startswith("SubClassOf"):
        line = line.strip()[11:-1]
        if not line:
          co-=1
          continue
        if not line.startswith("ObjectIntersectionOf(") and not line.startswith("ObjectSomeValuesFrom(") and line.find("ObjectSomeValuesFrom(") == -1:
          # SubClassOf C D
          it = line.split(' ')
          c = it[0]
          d = it[1]
          # subclass_axioms.append((og_line, c, d))
          if c not in subclass_classes:
            subclass_classes[c] = len(subclass_classes)
          if d not in subclass_classes:
            subclass_classes[d] = len(subclass_classes)

train = int(co*0.7)
valid = int(co*0.3)
test = co - train- valid
if test>0:
     valid+=test
     test=0
ex_train = train
total_train = train
count=0
print("total", co)
print("Expected train, valid, test:", train, valid, test)

  # Second pass: add all relation (rbox) axioms and only add non subclass axioms containing classes in subclass axioms
with open(file_path) as f:
    for line in f:
      og_line = line
      if line.startswith('SubObjectPropertyOf'):
        line = line.strip()[20:-1]
        if line.startswith('ObjectPropertyChain'):
          line_chain = line.strip()[20:-1]
          line1 = line.split(")")
          line10 = line1[0].split()
          if len(line10) < 2:
            continue
          r1 = line10[0]
          r2 = line10[1]
          r3 = line1[1]
          # if train and (r1 not in relations or r2 not in relations or r3 not in relations):
          file['train_norm'].write(og_line)
          file['train'].write(r1 + ' ' + r2 + '\n')
          count+=1
          train-=1
            # if r1 not in relations:
            #   relations[r1] = len(relations)
            # if r2 not in relations:
            #   relations[r2] = len(relations)
            # if r3 not in relations:
            #   relations[r3] = len(relations)
          # else:
            # other_axioms.append((og_line, r1, r2, r3))
        else:
          # print("Inside sub obj prop")
          it = line.split(' ')
          r1 = it[0]
          r2 = it[1]
          file['train_norm'].write(og_line)
          file['train'].write(r1 + ' ' + r2 + '\n')
          count+=1
          train-=1
          # other_axioms.append((og_line, r1, r2, ""))
        continue
      line = line.strip()[11:-1]
#           print(line)
      if not line:
        print(og_line)
        continue
      if line.startswith('ObjectIntersectionOf('):
        # C and D SubClassOf E
        it = line.split(' ')
        c = it[0][21:]
        d = it[1][:-1]
        e = it[2]
        if train and (c in subclass_classes or d in subclass_classes or e in subclass_classes):
          file['train_norm'].write(og_line)
          file['train'].write(c + ' ' + d + '\n')
          count+=1
          train-=1
          if c in subclass_classes:
            subclass_classes.pop(c)
          if d in subclass_classes:
            subclass_classes.pop(d)
          if e in subclass_classes:
            subclass_classes.pop(e)
          if c not in classes:
            classes[c] = len(classes)
          if d not in classes:
            classes[d] = len(classes)
          if e not in classes:
            classes[e] = len(classes)
        else:
          other_axioms.append((og_line, c, d, e))
      elif line.startswith('ObjectSomeValuesFrom('):
        # R some C SubClassOf D
        it = line.split(' ')
        r = it[0][21:]
        c = it[1][:-1]
        d = it[2]
        if train and (c in subclass_classes or d in subclass_classes or r not in relations):
          file['train_norm'].write(og_line)
          file['train'].write(c + ' ' + d + '\n')
          count+=1
          train-=1
          if c in subclass_classes:
            subclass_classes.pop(c)
          if d in subclass_classes:
            subclass_classes.pop(d)
          if c not in classes:
            classes[c] = len(classes)
          if d not in classes:
            classes[d] = len(classes)
          if r not in relations:
            relations[r] = len(relations)
        else:
          other_axioms.append((og_line, r, c, d))
      elif line.find('ObjectSomeValuesFrom') != -1:
        # C SubClassOf R some D
        it = line.split(' ')
        c = it[0]
        r = it[1][21:]
        d = it[2][:-1]
        if train and (c in subclass_classes or d in subclass_classes or r not in relations):
          file['train_norm'].write(og_line)
          file['train'].write(c + ' ' + d + '\n')
          count+=1
          train-=1
          if c in subclass_classes:
            subclass_classes.pop(c)
          if d in subclass_classes:
            subclass_classes.pop(d)
          if c not in classes:
            classes[c] = len(classes)
          if d not in classes:
            classes[d] = len(classes)
          if r not in relations:
            relations[r] = len(relations)
        else:
          other_axioms.append((og_line, r, c, d))
      else:
        # C SubClassOf D
        it = line.split(' ')
        c = it[0]
        d = it[1]
        subclass_axioms.append((og_line, c, d))
print("Added to train from other axioms", ex_train-train, count)
print(count+len(subclass_axioms)+len(other_axioms), len(subclass_axioms), len(other_axioms))
print("Classes not added", len([k for k in subclass_classes if k not in classes]), len(subclass_classes))
ex_train = train
# Second pass
temp = []
for tup in subclass_axioms:
    line, c, d = tup
    if train and (c in subclass_classes or d in subclass_classes):
#       print(line)
      file['train_norm'].write(line)
      file['train'].write(c + ' ' + d + '\n')
      count+=1
      train-=1
      if c in subclass_classes:
        subclass_classes.pop(c)
      if d in subclass_classes:
        subclass_classes.pop(d)
      if c not in classes:
        classes[c] = len(classes)
      if d not in classes:
        classes[d] = len(classes)
    else:
      temp.append((line, c, d))
subclass_axioms = temp
print(count+len(subclass_axioms)+len(other_axioms), len(subclass_axioms), len(other_axioms))
print("Added to train from subclass axioms", ex_train-train)
ex_train = train

print("remaining subclass axioms and other axioms", len(subclass_axioms), len(other_axioms))
print("train, valid, test", train, valid, test)

if len(subclass_axioms) < test+valid:
    # recalculate test, validation, training sample counts
    c = len(subclass_axioms)
    #valid = int(c*0.66)
    valid = int(c)
    #test = c - valid
   # if test>0:
    # valid+=test
     #test=0
    train = max(0, min(train, int(c*0.7/0.3) - (total_train-train)))
ex_train = train

print("Train, valid, test left", train, valid, test)

temp = []
while test and subclass_axioms != []:
    line, c, d = subclass_axioms.pop()
    if c not in subclass_classes and d not in subclass_classes:
      file['test'].write(c + ' ' + d + '\n')
      test-=1
      count+=1
    else:
      temp.append((line, c, d))
while valid and subclass_axioms != []:
    line, c, d =  subclass_axioms.pop()
    if c not in subclass_classes and d not in subclass_classes:
      file['valid'].write(c + ' ' + d + '\n')
      valid-=1
      count+=1
    else:
      temp.append((line, c, d))
subclass_axioms.extend(temp)
while train and other_axioms != []:
    line, r, c, d = other_axioms.pop()
#     if c in classes and d in classes:
    file['train'].write(c + ' ' + d + '\n')
    file['train_norm'].write(line)
    count+=1
    train-=1
print("added to train from other axioms", ex_train-train)
ex_train = train
while train and subclass_axioms != []:
    line, c, d =  subclass_axioms.pop()
#     if c in classes or d in classes:
    file['train'].write(c + ' ' + d + '\n')
    file['train_norm'].write(line)
    count+=1
    train -=1
print("added to train from subclass axioms", ex_train-train)
for k in ['train', 'valid', 'test']:
    file[k].close()
print("Total:", count)
print("left", len(subclass_axioms), len(other_axioms))
print(train, test, valid)

Running the above code gives two main file- **train.txt**, **valid.txt** among all. The **test.txt** file is also is also generated. It has no axioms/data as we don't need it.

The 3 files can be renamed and used in ElEm Algorithm.

Eg: for the below code we using **HPO17zt4f_data.zip**.

It containes **HPO17zt4f_data** folder which further contains

It consists of 1. **hasAnnotation42.owl**

2. **HPO_train17zt4f.txt** (train.txt renamed)

3. **HPO_valid17zt4f.txt** (valid.txt renamed)

3. **HPO_test17zt4f.txt** (test.txt renamed)

In the next part, the places which needs to be changed are marked by comment  **##CHANGE**. Please change it as per your use case.

# N-Ball Algorithm

In [ ]:
import os
os.mkdir('HPO17zt4f_results')  ##CHANGE
os.mkdir('HPO17zt4f_results/EmEL_dir')  ##CHANGE
!unzip 'HPO17zt4f_data.zip'  ##CHANGE
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import click as ck
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.framework import function
import re
import math
import matplotlib.pyplot as plt
import logging
from tensorflow.keras.layers import (
    Input,
)
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras import constraints
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from tensorflow.keras import backend as K
from scipy.stats import rankdata
from tensorflow.python.keras.utils.data_utils import Sequence




config = tf.compat.v1.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

logging.basicConfig(level=logging.INFO)

Archive:  HPO17zt4f_data.zip
   creating: HPO17zt4f_data/
  inflating: HPO17zt4f_data/HPO_test17zt4f.txt  
  inflating: HPO17zt4f_data/HPO_valid17zt4f.txt  
  inflating: HPO17zt4f_data/HPO_train17zt4f.txt  
  inflating: HPO17zt4f_data/readme17zt4f.txt  
  inflating: HPO17zt4f_data/hasAnnotation42.owl  


In [ ]:
def load_valid_data(valid_data_file, classes, relations):
    data = []
    rel = f'SubClassOf'
    with open(valid_data_file, 'r') as f:
        for line in f:
            it = line.strip().split()
            id1 = it[0]
            id2 = it[1]
            if id1 not in classes or id2 not in classes or rel not in relations:
                continue
            data.append((classes[id1], relations[rel], classes[id2]))
    return data

In [ ]:
def load_cls(train_data_file):
    train_subs=list()
    counter=0
    with open(train_data_file,'r') as f:
        for line in f:
            counter+=1
            it = line.strip().split()
            cls1 = it[0]
            cls2 = it[1]
            train_subs.append(cls1)
            train_subs.append(cls2)
    train_cls = list(set(train_subs))
    return train_cls,counter

In [ ]:
total_sub_cls=[]
train_file = "HPO17zt4f_data/HPO_train17zt4f.txt"  ##CHANGE
va_file = "HPO17zt4f_data/HPO_valid17zt4f.txt"   ##CHANGE
test_file = "HPO17zt4f_data/HPO_test17zt4f.txt"   ##CHANGE
train_sub_cls,train_samples = load_cls(train_file)
valid_sub_cls,valid_samples = load_cls(va_file)
test_sub_cls,test_samples = load_cls(test_file)
total_sub_cls = train_sub_cls + valid_sub_cls + test_sub_cls

all_subcls = list(set(total_sub_cls))
#print(len(all_subcls))


print("Training data samples:",train_samples)
print("Training data classes:",len(train_sub_cls))

Training data samples: 129432
Training data classes: 34387


In [ ]:
def load_data(filename):
    classes = {}
    relations = {}
    data = {'nf1': [], 'nf2': [], 'nf3': [], 'nf4': [], 'disjoint': []}
    with open(filename) as f:
        for line in f:
            # Ignore SubObjectPropertyOf
            if line.startswith('SubObjectPropertyOf'):
                #print("------->",line)
                line = line.strip()[20:-1]
                if "ObjectPropertyChain( )" not in line:
                    if line.startswith('ObjectPropertyChain'):
                        #print("A------->",line)
                        continue
            # Ignore SubClassOf()
            line = line.strip()[11:-1]
            if not line:
                continue
            if line.startswith('ObjectIntersectionOf(') :
              #if 'PATO' not in line:
                # C and D SubClassOf E
                it = line.split(' ')
                c = it[0][21:]
                d = it[1][:-1]
                e = it[2]
                if c not in classes:
                    classes[c] = len(classes)
                if d not in classes:
                    classes[d] = len(classes)
                if e not in classes:
                    classes[e] = len(classes)
                form = 'nf2'
                if e == 'owl:Nothing':
                    form = 'disjoint'
                data[form].append((classes[c], classes[d], classes[e]))

            elif line.startswith('ObjectSomeValuesFrom('):
                # R some C SubClassOf D
                it = line.split(' ')
                r = it[0][21:]
                c = it[1][:-1]
                d = it[2]
                if c not in classes:
                    classes[c] = len(classes)
                if d not in classes:
                    classes[d] = len(classes)
                if r not in relations:
                    relations[r] = len(relations)
                data['nf4'].append((relations[r], classes[c], classes[d]))
            elif line.find('ObjectSomeValuesFrom') != -1:
                # C SubClassOf R some D
                it = line.split(' ')
                c = it[0]
                r = it[1][21:]
                d = it[2][:-1]
                if c not in classes:
                    classes[c] = len(classes)
                if d not in classes:
                    classes[d] = len(classes)
                if r not in relations:
                    relations[r] = len(relations)
                data['nf3'].append((classes[c], relations[r], classes[d]))
            else:
                # C SubClassOf D
                it = line.split(' ')
                c = it[0]
                d = it[1]
                r = 'SubClassOf'
                if r not in relations:
                    relations[r] = len(relations)
                if c not in classes:
                    classes[c] = len(classes)
                if d not in classes:
                    classes[d] = len(classes)
                data['nf1'].append((classes[c],relations[r],classes[d]))

    # Check if TOP in classes and insert if it is not there
    if 'owl:Thing' not in classes:
        classes['owl:Thing'] = len(classes)
#changing by adding sub classes of train_data ids to prot_ids
    prot_ids = []
    class_keys = list(classes.keys())
    for val in all_subcls:
        if val not in class_keys:
            cid = len(classes)
            classes[val] = cid
            prot_ids.append(cid)
        else:
            prot_ids.append(classes[val])
#     for k, v in classes.items():
#         if k in all_subcls:
#             prot_ids.append(v)

    prot_ids = np.array(prot_ids)
    prot_ids2=np.array(list(relations.values()))

    # Add corrupted triples nf3
    n_classes = len(classes)
    data['nf_neg'] = []
    for c, r, d in data['nf1']:
        x = np.random.choice(prot_ids)
        while x == c:
            x = np.random.choice(prot_ids)

        y = np.random.choice(prot_ids)
        while y == d:
             y = np.random.choice(prot_ids)
        data['nf_neg'].append((c, r,x))
        data['nf_neg'].append((y, r, d))

    for c, r, d in data['nf3']:
        x = np.random.choice(prot_ids)
        while x == c:
            x = np.random.choice(prot_ids)

        y = np.random.choice(prot_ids)
        while y == d:
             y = np.random.choice(prot_ids)
        data['nf_neg'].append((c, r,x))
        data['nf_neg'].append((y, r, d))

    data['radius'] = []
    for val in classes:
        data['radius'].append(classes[val])


    data['nf1'] = np.array(data['nf1'])
    data['nf2'] = np.array(data['nf2'])
    data['nf3'] = np.array(data['nf3'])
    data['nf4'] = np.array(data['nf4'])
    data['disjoint'] = np.array(data['disjoint'])
    data['top'] = np.array([classes['owl:Thing'],])
    data['nf_neg'] = np.array(data['nf_neg'])
    data['radius'] = np.array(data['radius'])


    for key, val in data.items():
        index = np.arange(len(data[key]))
        np.random.seed(seed=100)
        np.random.shuffle(index)
        data[key] = val[index]

    return data, classes, relations

In [ ]:
class ELModel(tf.keras.Model):

    def __init__(self, nb_classes, nb_relations, embedding_size, batch_size, margin, reg_norm=1):
        super(ELModel, self).__init__()
        self.nb_classes = nb_classes
        self.nb_relations = nb_relations
        self.margin = margin
        self.reg_norm = reg_norm
        self.batch_size = batch_size
        self.inf = 5.0 # For top radius #100
        #initialization of class weights and radius
        csample_weights = np.random.uniform(low=-1, high=1, size=(nb_classes, embedding_size))
        radius_wts = np.random.uniform(low=0, high=1, size=(nb_classes,))
        cls_weights = np.column_stack((csample_weights,radius_wts))
        cls_weights = cls_weights / np.linalg.norm(
            cls_weights, axis=1).reshape(-1, 1)
        rel_weights = np.random.uniform(low=-1, high=1, size=(nb_relations, embedding_size))
        rel_weights = rel_weights / np.linalg.norm(
            rel_weights, axis=1).reshape(-1, 1)
        self.cls_embeddings = tf.keras.layers.Embedding(
            nb_classes,
            embedding_size + 1,
            input_length=1,
            weights=[cls_weights,])
        self.rel_embeddings = tf.keras.layers.Embedding(
            nb_relations,
            embedding_size,
            input_length=1,
            weights=[rel_weights,])


    def call(self, input):
        """Run the model."""

        nf1, nf2, nf3, nf4,dis,top, nf_neg,radius = input
        loss1 = self.nf1_loss(nf1)
        loss2 = self.nf2_loss(nf2)
        loss3 = self.nf3_loss(nf3)
        loss4 = self.nf4_loss(nf4)
        loss_dis = self.dis_loss(dis)
        loss_top = self.top_loss(top)
        loss_nf_neg = self.nf_neg_loss(nf_neg)
        loss5 = self.radius_loss(radius)
        loss = loss1 +loss3 + loss4 +loss_dis+loss_top+loss_nf_neg - loss5
        return loss


    def reg(self, x):
        res = tf.abs(tf.norm(x, axis=1) - self.reg_norm)
        res = tf.reshape(res, [-1, 1])
        return res

    def nf1_loss(self, input):
        c = input[:, 0]
        r = input[:,1]
        d = input[:, 2]
        c = self.cls_embeddings(c)
        d = self.cls_embeddings(d)
        r = self.rel_embeddings(r)

        rc = tf.math.maximum(0.0,c[:, -1])
        rd = tf.math.maximum(0.0,d[:, -1])
        x1 = c[:, 0:-1]
        x2 = d[:, 0:-1]
        x3 = x1 + r
        euc = tf.norm(x3 - x2, axis=1)
        dst = tf.reshape(tf.nn.relu(euc + rc - rd - self.margin), [-1, 1])
        return dst + self.reg(x1) + self.reg(x2)

    def nf2_loss(self, input):
        c = input[:, 0]
        d = input[:, 1]
        e = input[:, 2]
        c = self.cls_embeddings(c)
        d = self.cls_embeddings(d)
        e = self.cls_embeddings(e)
        rc = tf.reshape(tf.math.maximum(0.0,c[:, -1]), [-1, 1])
        rd = tf.reshape(tf.math.maximum(0.0,d[:, -1]), [-1, 1])
        re = tf.reshape(tf.math.maximum(0.0,e[:, -1]), [-1, 1])
        sr = rc + rd
        x1 = c[:, 0:-1]
        x2 = d[:, 0:-1]
        x3 = e[:, 0:-1]

        x = x2 - x1
        dst = tf.reshape(tf.norm(x, axis=1), [-1, 1])
        dst2 = tf.reshape(tf.norm(x3 - x1, axis=1), [-1, 1])
        dst3 = tf.reshape(tf.norm(x3 - x2, axis=1), [-1, 1])
        rdst = tf.nn.relu(tf.math.minimum(rc, rd) - re - self.margin)

        dst_loss=((tf.nn.relu(tf.math.log(dst)-tf.math.log(sr)-self.margin))
        +tf.nn.relu(tf.nn.relu(tf.math.log(dst2)-tf.math.log(rc)-self.margin))
        +tf.nn.relu(tf.nn.relu(tf.math.log(dst3)-tf.math.log(rd)-self.margin))+rdst)
        return dst_loss + self.reg(x1) + self.reg(x2) + self.reg(x3)

    def nf3_loss(self, input):
        # C subClassOf R some D
        c = input[:, 0]
        r = input[:, 1]
        d = input[:, 2]
        c = self.cls_embeddings(c)
        d = self.cls_embeddings(d)
        r = self.rel_embeddings(r)
        x1 = c[:, 0:-1]
        x2 = d[:, 0:-1]
        x3 = x1 + r

        rc = tf.math.maximum(0.0,c[:, -1])
        rd = tf.math.maximum(0.0,d[:, -1])
        euc = tf.norm(x3 - x2, axis=1)
        dst = tf.reshape(tf.nn.relu(euc + rc - rd - self.margin), [-1, 1])

        return dst + self.reg(x1) + self.reg(x2)

    def nf_neg_loss(self, input):
        # C subClassOf R some D
        c = input[:, 0]
        r = input[:, 1]
        d = input[:, 2]
        c = self.cls_embeddings(c)
        d = self.cls_embeddings(d)
        r = self.rel_embeddings(r)
        x1 = c[:, 0:-1]
        x2 = d[:, 0:-1]

        x3 = x1 + r

        rc = tf.math.maximum(0.0,c[:, -1])
        rd = tf.math.maximum(0.0,d[:, -1])
        euc = tf.norm(x3 - x2, axis=1)
        dst = tf.reshape((-(euc - rc - rd) + self.margin), [-1, 1])

        return dst + self.reg(x1) + self.reg(x2)

    def nf4_loss(self, input):
        # R some C subClassOf D
        r = input[:, 0]
        c = input[:, 1]
        d = input[:, 2]
        c = self.cls_embeddings(c)
        d = self.cls_embeddings(d)
        r = self.rel_embeddings(r)
        rc = tf.reshape(tf.math.maximum(0.0,c[:, -1]), [-1, 1])
        rd = tf.reshape(tf.math.maximum(0.0,d[:, -1]), [-1, 1])
        sr = rc + rd
        x1 = c[:, 0:-1]
        x2 = d[:, 0:-1]

        # c - r should intersect with d
        x3 = x1 - r
        dst = tf.reshape(tf.norm(x3 - x2, axis=1), [-1, 1])
        dst_loss = tf.nn.relu(dst - sr - self.margin)
        return dst_loss + self.reg(x1) + self.reg(x2)


    def dis_loss(self, input):
        c = input[:, 0]
        d = input[:, 1]
        c = self.cls_embeddings(c)
        d = self.cls_embeddings(d)
        rc = tf.reshape(tf.math.maximum(0.0,c[:, -1]), [-1, 1])
        rd = tf.reshape(tf.math.maximum(0.0,d[:, -1]), [-1, 1])
        sr = rc + rd
        x1 = c[:, 0:-1]
        x2 = d[:, 0:-1]

        dst = tf.reshape(tf.norm(x2 - x1, axis=1), [-1, 1])
        return tf.nn.relu(sr - dst + self.margin) + self.reg(x1) + self.reg(x2)


    def top_loss(self, input):
        d = input[:, 0]
        d = self.cls_embeddings(d)
        rd = tf.reshape(tf.math.maximum(0.0,d[:, -1]), [-1, 1])
        return tf.math.abs(rd - self.inf)

    def inclusion_loss(self,input):
        r1 = input[:, 0]
        r2 = input[:, 1]
        r1 = self.rel_embeddings(r1)
        r2 = self.rel_embeddings(r2)
        #print("r2 type------>",type(r2))

        euc = tf.norm(r2 - r1, axis=1)

        normalize_a = tf.nn.l2_normalize(r1,0)
        normalize_b = tf.nn.l2_normalize(r2,0)
        direction=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
        dir_loss = tf.abs(1 - direction)
        dir_loss = tf.reshape(dir_loss, [-1, 1])
        dst = tf.reshape(tf.nn.relu(euc - self.margin), [-1, 1])

        return dst + self.reg(r1) + self.reg(r2) + dir_loss

    def chain_loss(self,input):
        r1 = input[:, 0]
        r2 = input[:, 1]
        r3 = input[:, 2]
        c = self.rel_embeddings(r1)
        d = self.rel_embeddings(r2)
        e = self.rel_embeddings(r3)
        s = e - c

        dst = tf.reshape(tf.norm(c - d, axis=1), [-1, 1])
        dst2 = tf.reshape(tf.norm(e - c, axis=1), [-1, 1])
        dst3 = tf.reshape(tf.norm(e - d, axis=1), [-1, 1])
        dst_dir = tf.reshape(tf.norm(s - d, axis=1), [-1, 1])
        dst_loss = (tf.nn.relu(dst_dir - self.margin))
        res_vec = c + d
        normalize_a = tf.nn.l2_normalize(res_vec,0)
        normalize_b = tf.nn.l2_normalize(e,0)
        direction=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
        dir_loss = tf.abs(1 - direction)
        dir_loss = tf.reshape(dir_loss, [-1, 1])
        return dst_loss + self.reg(c) + self.reg(d) + self.reg(e) + dir_loss

    def radius_loss(self, input):
        d = input[:, 0]
        d = self.cls_embeddings(d)
        rd = tf.reshape(d[:, -1], [-1, 1])
        return tf.math.minimum(0.0,rd)


# In[10]:


class Generator(object):

    def __init__(self, data, batch_size, steps=80):  #100
        self.data = data
        self.batch_size = batch_size
        self.steps = steps
        self.start = 0

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()

    def reset(self):
        self.start = 0

    def next(self):
        if self.start < self.steps:
            nf1_index = np.random.choice(
                self.data['nf1'].shape[0], self.batch_size)
            nf2_index = np.random.choice(
                self.data['nf2'].shape[0], self.batch_size)
            nf3_index = np.random.choice(
                self.data['nf3'].shape[0], self.batch_size)
            nf4_index = np.random.choice(
                self.data['nf4'].shape[0], self.batch_size)
            dis_index = np.random.choice(
                self.data['disjoint'].shape[0], self.batch_size)
            top_index = np.random.choice(
                self.data['top'].shape[0], self.batch_size)
            nf_neg_index = np.random.choice(
                self.data['nf_neg'].shape[0], self.batch_size)
            radius_index = np.random.choice(
                self.data['radius'].shape[0], self.batch_size)
            nf1 = self.data['nf1'][nf1_index]
            nf2 = self.data['nf2'][nf2_index]
            nf3 = self.data['nf3'][nf3_index]
            nf4 = self.data['nf4'][nf4_index]
            dis = self.data['disjoint'][dis_index]
            top = self.data['top'][top_index]
            nf_neg = self.data['nf_neg'][nf_neg_index]
            radius = self.data['radius'][radius_index]
            labels = np.zeros((self.batch_size, 1), dtype=np.float32)
            self.start += 1
            return ([nf1, nf2, nf3, nf4,dis,top,nf_neg,radius], labels)
        else:
            self.reset()


# In[11]:


class MyModelCheckpoint(ModelCheckpoint):

    def __init__(self, *args, **kwargs):
        super(ModelCheckpoint, self).__init__()
        self.out_classes_file = kwargs.pop('out_classes_file')
        self.out_relations_file = kwargs.pop('out_relations_file')
        self.monitor = kwargs.pop('monitor')
        self.cls_list = kwargs.pop('cls_list')
        self.rel_list = kwargs.pop('rel_list')
        self.load_weights_on_restart = False
        ##
        self.save_freq='epoch'
        ##
        self.valid_data = kwargs.pop('valid_data')
        self.proteins = kwargs.pop('proteins')
        self.prot_index = list(self.proteins.values())
        self.prot_dict = {v: k for k, v in enumerate(self.prot_index)}

        self.best_rank = 100000

    def on_epoch_end(self, epoch, logs=None):
        # Save embeddings every 10 epochs
        current_loss = logs.get(self.monitor)
        if math.isnan(current_loss):
            print('NAN loss, stopping training')
            self.model.stop_training = True
            return
        el_model = self.model.layers[-1]
        cls_embeddings = el_model.cls_embeddings.get_weights()[0]
        rel_embeddings = el_model.rel_embeddings.get_weights()[0]

        prot_embeds = cls_embeddings[self.prot_index]
        prot_rs = prot_embeds[:, -1].reshape(-1, 1)
        prot_embeds = prot_embeds[:, :-1]
        mean_rank = 0
        n = len(self.valid_data)

        for c, r, d in self.valid_data:
            c, r, d = self.prot_dict[c], r, self.prot_dict[d]
            ec = prot_embeds[c, :]
            rc = prot_rs[c, :]
            er = rel_embeddings[r, :]
            ec += er

            dst = np.linalg.norm(prot_embeds - ec.reshape(1, -1), axis=1)
            dst = dst.reshape(-1, 1)
            res = np.maximum(0, dst - rc - prot_rs - el_model.margin)
            res = res.flatten()
            index = rankdata(res, method='average')
            rank = index[d]
            mean_rank += rank

        mean_rank /= n
        # fmean_rank /= n
        print(f'\n Validation {epoch + 1} {mean_rank}\n')

        if mean_rank < self.best_rank:
            self.best_rank = mean_rank
            print(f'\n Saving embeddings {epoch + 1} {mean_rank}\n')
            cls_file = self.out_classes_file
            rel_file = self.out_relations_file


            df = pd.DataFrame(
                {'classes': self.cls_list, 'embeddings': list(cls_embeddings)})
            #print(df.head(1))
            df.to_pickle(cls_file)


            df = pd.DataFrame(
                {'relations': self.rel_list, 'embeddings': list(rel_embeddings)})
            df.to_pickle(rel_file)
        return epoch



def build_model(device,train_data,classes,relations,valid_data):
    proteins = {}#substitute for classes with subclass case
    for val in all_subcls:
        proteins[val] = classes[val]
    nb_classes = len(classes)
    nb_relations = len(relations)
    print("no. classes:",nb_classes)
    print("no. relations:",nb_relations)
    nb_train_data = 0
    for key, val in train_data.items():
        nb_train_data = max(len(val), nb_train_data)
    train_steps = int(math.ceil(nb_train_data / (1.0 * batch_size)))
    train_generator = Generator(train_data, batch_size, steps=train_steps)

    cls_dict = {v: k for k, v in classes.items()}
    rel_dict = {v: k for k, v in relations.items()}

    cls_list = []
    rel_list = []
    for i in range(nb_classes):
        cls_list.append(cls_dict[i])
    for i in range(nb_relations):
        rel_list.append(rel_dict[i])

    with tf.device('/' + device):
        nf1 = Input(shape=(3,), dtype=np.int32)
        nf2 = Input(shape=(3,), dtype=np.int32)
        nf3 = Input(shape=(3,), dtype=np.int32)
        nf4 = Input(shape=(3,), dtype=np.int32)
        dis = Input(shape=(3,), dtype=np.int32)
        top = Input(shape=(1,), dtype=np.int32)
        nf_neg = Input(shape=(3,), dtype=np.int32)
        radius = Input(shape=(1,), dtype=np.int32)
        el_model = ELModel(nb_classes, nb_relations, embedding_size, batch_size, margin, reg_norm)
        out = el_model(input=[nf1,nf2, nf3, nf4,dis,top, nf_neg,radius])
        model = tf.keras.Model(inputs=[nf1,nf2, nf3, nf4, dis,top,nf_neg,radius], outputs=out)

        optimizer = optimizers.Adam(lr=learning_rate)


        model.compile(optimizer=optimizer, loss="mse")

    # TOP Embedding
        top = classes.get('owl:Thing', None)
        checkpointer = MyModelCheckpoint(
            out_classes_file=out_classes_file,
            out_relations_file=out_relations_file,
            cls_list=cls_list,
            rel_list=rel_list,
            valid_data=valid_data[0:100],
            proteins=proteins,
            monitor='loss')


        logger = CSVLogger(loss_history_file)

        # Save initial embeddings
        cls_embeddings = el_model.cls_embeddings.get_weights()[0]
        rel_embeddings = el_model.rel_embeddings.get_weights()[0]

        cls_file = out_classes_file
        rel_file = out_relations_file


        df = pd.DataFrame(
            {'classes': cls_list, 'embeddings': list(cls_embeddings)})
        df.to_pickle(cls_file)
        df = pd.DataFrame(
            {'relations': rel_list, 'embeddings': list(rel_embeddings)})
        df.to_pickle(rel_file)

        model.fit_generator(
            train_generator,
            steps_per_epoch=train_steps,
            epochs=epochs,
            workers=12,
            callbacks=[logger, checkpointer])


# In[13]:



gdata_file="HPO17zt4f_data/hasAnnotation42.owl"  ##CHANGE
train_data_model, classes, relations = load_data(gdata_file)


# In[14]:


valid_data_file="HPO17zt4f_data/HPO_valid17zt4f.txt"  ##CHANGE
valid_data_model = load_valid_data(valid_data_file, classes, relations)



margins = [-0.1]  ##CHANGE
embed_dims = [100]  ##CHANGE
batch_size =   256
device='gpu:0'
reg_norm=1
learning_rate= 3e-4
epochs=1000
for d in embed_dims:
    embedding_size = d
    print("***************Embedding Dim:",embedding_size,'****************')
    for m in margins:
        margin = m
        print("**************Margin Loss:",margin,"***************")
        out_classes_file = f'HPO17zt4f_results/EmEL_dir/HPO17zt4f_{embedding_size}_{margin}_{epochs}_cls.pkl'  ##CHANGE
        out_relations_file = f'HPO17zt4f_results/EmEL_dir/HPO17zt4f_{embedding_size}_{margin}_{epochs}_rel.pkl'  ##CHANGE
        loss_history_file= f'HPO17zt4f_results/EmEL_dir/HPO17zt4f_lossHis_{embedding_size}_{margin}_{epochs}.csv'  ##CHANGE
        build_model(device,train_data_model,classes,relations,valid_data_model)